# Assertion RLA

**From 4 October 2022, major re-write. not fully tested.**

**From 18 May 2022, consistent sampling seems to be working.**

**From 1 April 2022, integrating alpha_mart, p_history, and other features.**

**From 24 October 2021, dev version implementing consistent sampling to target smaller contests.**

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers for each contest
    - an upper bound on the number of ballot cards that contain each contest
    - an upper bound on the total number of cards across all contests
    - whether to use card style information to target sampling
+ a ballot manifest (see below)
+ a random seed
+ a file of cast vote records
+ reported results for each contest
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper cards selected for audit

`use_style` controls whether the sample is drawn from all cards (`use_style == False`) or card style information is used
to target the cards that purport to contain each contest (`use_style == True`).
In the current implementation, card style information is inferred from cast-vote records, with additional 'phantom' CVRs if there could be more cards that contain a contest than is accounted for in the CVRs.
Errors in the card style information are treated conservatively using the  "phantoms-to-evil-zombies" (~2EZ) approach ([Banuelos & Stark, 2012](https://arxiv.org/abs/1207.3413)) so that the risk limit remains valid, even if the CVRs misrepresent
which cards contain which contests.

The two ways of sampling are treated differently. 
If the sample is to be drawn only from cards that--according to the CVR--contain a particular contest, and a sampled card turns out not to
contain that contest, that is considered a discrepancy, dealt with using the ~2EZ approach.
It is assumed that every CVR corresponds to a card in the manifest, but there might
be cards cast in the contest for which there is no corresponding CVR. In that case,
phantom CVRs are created to ensure that the audit is still truly risk-limiting.

Given an independent (i.e., not relying on the voting system) upper bound on the number of cards that contain the contest, if the number of CVRs that contain the contest does not exceed that bound, we can sample from paper purported to contain the contest and use the ~2EZ approach to deal with missing CVRs. This can greatly increase the efficiency of the audit if 
some contests appear on only a small percentage of the cast cards ([Glazer, Spertus, and Stark, 2021](https://dl.acm.org/doi/10.1145/3457907)).
If there are more CVRs than the upper bound on the number of cards, extra CVRs can be deleted provided
that deletion does not change any contest outcome. See [Stark, 2022](https://arxiv.org/abs/2207.01362).
(However, if there more CVRs than cards, that is evidence of a process failure.)

Any sampled phantom card (i.e., a card for which there is no CVR) is treated as if its CVR is a non-vote (which it is), and as if its MVR was least favorable (an "evil zombie" producing the greatest doubt in every assertion, separately). Any sampled card for which there is a CVR is compared to its corresponding CVR. 
If the card turns out not to contain the contest (despite the fact that the CVR says it does), the MVR is treated in the least favorable way for each assertion (i.e., as a zombie rather than as a non-vote).

The tool helps select cards for audit, and reports when the audit has found sufficiently strong evidence to stop.

The tool exports a log of all the audit inputs except the CVR file, but including the auditors' manually determined voter intent from the audited cards.

The pre-10/2021 version used a single sample to audit all contests. 

### Internal workflow

+ Read overall audit information (including the seed) and contest information
+ Read assertions for IRV contests and construct assertions for all other contests
+ Read ballot manifest
+ Read cvrs. Every CVR should have a corresponding manifest entry.
+ Prepare ~2EZ:
    - `N_phantoms = max_cards - cards_in_manifest`
    - If `N_phantoms < 0`, complain
    - Else create `N_phantoms` phantom cards
    - For each contest `c`:
        + `N_c` is the input upper bound on the number of cards that contain `c`
        + if `N_c is None`, `N_c = max_cards - non_c_cvrs`, where `non_c_cvrs` is #CVRs that don't contain `c`
        + `C_c` is the number of CVRs that contain the contest
        + if `C_c > N_c`, complain
        + else if `N_c - C_c > N_phantoms`, complain
        + else:
            - Consider contest `c` to be on the first `N_c - C_c` phantom CVRs
            - Consider contest `c` to be on the first `N_c - C_c` phantom ballots
+ Create Assertions for every Contest. This involves also creating an Assorter for every Assertion, and a `NonnegMean` test
for every Assertion.
+ Calculate assorter margins for all assorters:
    - If `not use_style`, apply the Assorter to all cards and CVRs, including phantoms
    - Else apply the assorter only to cards/cvrs reported to contain the contest, including phantoms that contain the contest
+ Set `assertion.test.u` to the appropriate value for each assertion: `assorter.upper_bound` for polling audits or 
      `2/(2-assorter.margin/assorter.upper_bound)`
+ Estimate starting sample size for the specified sampling design (w/ or w/o replacement, stratified, etc.), for chosen risk function, use of card-style information, etc.:
    - User-specified criterion, controlled by parameters. Examples:
        + expected sample size for completion, on the assumption that there are no errors
        + 90th percentile of sample size for completion, on the assumption that errors are not more frequent than specified
    - If `not use_style`, base estimate on sampling from the entire manifest, i.e., smallest assorter margin
    - Else use consistent sampling:
        + Augment each CVR (including phantoms) with a probability of selection, `p`, initially 0
        + For each contest `c`:
            - Find sample size `n_c` that meets the criterion 
            - For each non-phantom CVR that contains the contest, set `p = max(p, n_c/N_c)` 
        + Estimated sample size is the sum of `p` over all non-phantom CVRs
+ Draw the random sample:
    - Use the specified design, including using consistent sampling for style information
    - Express sample cards in terms of the manifest
    - Export
+ Read manual interpretations of the cards (MVRs)
+ Calculate attained risk for each assorter
    - Use ~2EZ to deal with phantom CVRs or cards; the treatment depends on whether `use_style == True`
+ Report
+ Estimate incremental sample size if any assorter nulls have not been rejected
+ Draw incremental sample; etc

In [ ]:
import math
import json
import warnings
import numpy as np
import pandas as pd
import csv
import copy

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256, int_from_hash
from cryptorandom.sample import sample_by_index

from Utils import Utils
from Audit import Audit, Assertion, Assorter, Contest
from CVR import CVR
from NonnegMean import NonnegMean
from dominion_tools import \
    prep_manifest, sample_from_cvrs, sample_from_manifest, write_cards_sampled

# Audit parameters.

The overall audit involves information that is the same across contests, encapsulated in
a dict called `audit`:

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample 
* `quantile`: quantile of the sample size to use for setting initial sample size
* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `use_style`: Boolean. If True, use card style information (inferred from CVRs) to target samples. If False, sample from all cards, regardless of the contest.
* `sample_file`: filename for sampled card identifiers (output)
* `mvr_file`: filename for manually ascertained votes from sampled cards (input)
* `log_file`: filename for audit log (output)
* `error_rate_1`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude after a single round even if the audit finds errors
* `error_rate_2`: expected rate of 2-vote overstatements. Recommended value $\ge$ 0.0001. Larger values increase the initial sample size, but make it more likely that the audit will conclude after a single round even if the audit finds errors
* `reps`: number of replications to use to estimate sample sizes. If `reps is None`, uses a deterministic method
* `quantile`: quantile of sample size to estimate. Not used if `reps is None`
* `strata`: a dict describing the strata. Keys are stratum identifiers; values are dicts containing:
    + `max_cards`: an upper bound on the number of pieces of paper cast in the contest. This should be derived independently of the voting system. A ballot consists of one or more cards.
    + `replacement`: whether to sample from this stratum with replacement. 
    + `use_style`: True if the sample in that stratum uses card-style information.
    + `audit_type` one of Contest.POLLING, Contest.BALLOT_COMPARISON, Contest.BATCH_COMPARISON but only POLLING and BALLOT_COMPARISON are currently implemented. 
    + `test`: the name of the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_mart`, `alpha_mart`. 
Not all risk functions work with every social choice function or every sampling method. 
    + `estimator`: the estimator to be used by the risk function. Options are [FIX ME!]
    + `test_kwargs`: keyword arguments for the risk function

----

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are Contest objects with attributes:
        - `risk_limit`: the risk limit for the audit of this contest
        - `cards`: an upper bound on the number of cast cards that contain the contest
        - `choice_function`: `Audit.SOCIAL_CHOICE_FUNCTION.PLURALITY`, 
          `Audit.SOCIAL_CHOICE_FUNCTION.SUPERMAJORITY`, or `Audit.SOCIAL_CHOICE_FUNCTION.IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV not supported)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3. share_to_win*n_winners must be less than 100%)
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won. Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions
        - `audit_type`: the audit strategy. Currently `Audit.AUDIT_TYPE.POLLING (ballot-polling)` and 
           `Audit.AUDIT_TYPE.BALLOT_COMPARISON` (ballot-level comparison audits) are implemented. 
           HYBRID and STRATIFIED are planned.
        - `test`: the risk function for the audit. Default is `NonnegMean.alpha_mart`, the alpha supermartingale test
        - `estim`: estimator for the alternative hypothesis for the test. Default is `NonnegMean.shrink_trunc`
        - `use_style`: True to use style information from CVRs to target the sample. False for polling audits or for sampling from all ballots for every contest.
        - other keys and values are added by the software, including `cvrs`, the number of CVRs that contain the contest, and `p`, the sampling fraction expected to be required to confirm the contest

In [ ]:
audit = {
         'seed':           12345678901234567890,
         'cvr_file':       './Data/SFDA2019_PrelimReport12VBMJustDASheets.raire',
         'manifest_file':  './Data/N19 ballot manifest with WH location for RLA Upload VBM 11-14.xlsx',
         'sample_file':    './Data/sample.csv',
         'mvr_file':       './Data/mvr.json',
         'log_file':       './Data/log.json',
         'quantile':       0.8,
         'error_rate_1':   0.001,
         'error_rate_2':   0.0001,
         'reps':           100,
         'strata':         {'stratum_1': {'max_cards':   293555, 
                                          'use_style':   True,
                                          'replacement': True,
                                          'audit_type':  Audit.AUDIT_TYPE.BALLOT_COMPARISON,
                                          'test':        NonnegMean.alpha_mart,
                                          'estimator':   NonnegMean.optimal_comparison,
                                          'test_kwargs': {}
                                         }
                           }
        }

# max_cards = 293555 # 146662 VBM turnout per SF Elections release 12 https://sfelections.sfgov.org/november-5-2019-election-results-summary

In [ ]:
cvr_file = './Data/SFDA2019_PrelimReport12VBMJustDASheets.raire'
manifest_file = './Data/N19 ballot manifest with WH location for RLA Upload VBM 11-14.xlsx'
sample_file = './Data/sample.csv'
# mvr_file = './Data/mvr_prepilot_test.json'
# mvr_file = './Data/mvrTest-PR12-DA-VBM-AllBallots-4TargetedErrors.json'
mvr_file = './Data/mvr.json'
log_file = './Data/log.json'

In [ ]:
error_rate_1 = 0.002  # rate of 1-vote overstatements, for sample size estimation
error_rate_2 = 0.0002 # rate of 2-vote overstatements, for sample size estimation
quantile = 0.5        # quantile of sample size to use for initial sample size
reps = int(10**2)     # replications to estimate sample size

In [ ]:
# contests to audit. Edit with details of your contest (eg., Contest 339 is the DA race)
contest_dict = {'339':{
                   'name': 'DA',
                   'risk_limit':       0.05,
                   'cards':            146662,
                   'choice_function':  Audit.SOCIAL_CHOICE_FUNCTION.IRV,
                   'n_winners':        1,
                   'candidates':       ['15','16','17','18'],
                   'reported_winners': ['15'],
                   'assertion_file':   './Data/SF2019Nov8Assertions.json',
                   'audit_type':       Audit.AUDIT_TYPE.BALLOT_COMPARISON,
                   'test':             NonnegMean.alpha_mart,
                   'estim':            NonnegMean.optimal_comparison,
                   'use_style':        True
                  }
               }

contests = Contests.from_dict(contest_dict)

Example of other social choice functions:

        contests =  {'city_council':{'name': 'City Council',
                             'risk_limit':0.05,
                             'cards': None,
                             'choice_function': Contest.PLURALITY,
                             'n_winners':3,
                             'candidates':['Doug','Emily','Frank','Gail','Harry'],
                             'reported_winners' : ['Doug', 'Emily', 'Frank']
                            },
                        'measure_1':{'name': 'Measure 1',
                             'risk_limit':0.05,
                             'cards': 65432,
                             'choice_function': Contest.SUPERMAJORITY,
                             'share_to_win':2/3,
                             'n_winners':1,
                             'candidates':['yes','no'],
                             'reported_winners' : ['yes']
                            }                  
                      }
              
If `cards` is `None`, uses `max_cards` as the upper bound for the contest.

In [ ]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c].choice_function == Audit.SOCIAL_CHOICE_FUNCTION.IRV:
        with open(contests[c].assertion_file, 'r') as f:
            contests[c].assertion_json = json.load(f)['audits'][0]['assertions']

In [ ]:
# construct the dict of dicts of assertions for each contest
Assertion.make_all_assertions(contests)

In [ ]:
for c in contests:
    print(f'{c}\n{contests[c]["assertions"]}')

## Read the ballot manifest

In [ ]:
# special for Primary/Dominion manifest format
manifest = pd.read_excel(manifest_file)

## Read the CVRs 

In [ ]:
# for ballot-level comparison audits

cvr_input = []
with open(cvr_file) as f:
    cvr_reader = csv.reader(f, delimiter=',', quotechar='"')
    for row in cvr_reader:
        cvr_input.append(row)

print("Read {} rows".format(len(cvr_input)))

In [ ]:
# Import/convert CVRs
cvr_list = CVR.from_raire(cvr_input)
print("After merging, there are CVRs for {} cards".format(len(cvr_list)))

In [ ]:
# turn RAIRE-style card identifiers into Dominion's style by substituting "-" for "_"
for c in cvr_list:
    c.id = (str(c.id).replace("_","-"))

In [ ]:
for i in range(10):
    print(str(cvr_list[i]))

In [ ]:
contests

In [ ]:
# check whether the manifest accounts for every card
max_cards, np.sum(manifest['Total Ballots'])

In [ ]:
# Check that there is a card in the manifest for every card (possibly) cast. If not, add phantoms.
manifest, manifest_cards, phantom_cards = prep_manifest(manifest, max_cards, len(cvr_list))
manifest

## Create CVRs for phantom cards

In [ ]:
# For Comparison Audits Only
#----------------------------

# If the sample draws a phantom card, these CVRs will be used in the comparison.
# phantom MVRs should be treated as zeros by the Assorter for every contest

# setting use_style = False to generate phantoms

cvr_list, phantom_vrs = CVR.make_phantoms(max_cards, cvr_list, contests, use_style=use_style, prefix='phantom-1-')
print(f"Created {phantom_vrs} phantom records")

In [ ]:
contests

In [ ]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
min_margin = Assertion.set_all_margins(contests, cvr_list, use_style=use_style)

print("minimum assorter margin {}".format(min_margin))
for c in contests:
    print("margins in contest {}".format(c))
    for a, m in contests[c]['margins'].items():
        print(a, m)

In [ ]:
# FIX ME! This needs to be revised for the new data structure
# Utils.check_audit_parameters(risk_function, error_rate, contests)

In [ ]:
# print(f'{log_file=}\n{seed=}\n{replacement=}\n{risk_function=}\n{g=}\n{max_cards=}'
#      f'\n{len(cvr_list)=}\n{manifest_cards=}\n{phantom_cards=}\n{error_rate=}\n{contests=}')

In [ ]:
Utils.write_audit_parameters(log_file, seed, replacement, risk_function, max_cards, len(cvr_list), \
                      manifest_cards, phantom_cards, error_rate, contests)

## Set up for sampling

## Find initial sample size

In [ ]:
# find initial sample size 
rf_vec = lambda x, N, t, **kwargs: risk_fn(x, N, t, **kwargs)[1]   # p is first returned value; p_history is the second returned value
                                                     # initial_sample_size() uses the whole history vector

ss_fn = lambda r, N, t, N, **kwargs: NonnegMean.initial_sample_size(
                        risk_function=rf_vec, x = None, N=N, t=1/2, polling=False,
                        error_rate=error_rate, alpha=r, u=u, reps=50,
                        bias_up=True, quantile=quantile) # change for polling audits

# debugging
print(NonnegMean.initial_sample_size(risk_function=rf, N=len(cvr_list), margin=0.01, \
                            polling=False, error_rate=0.001, alpha=0.05, \
                            t=1/2, u=1, reps=None,\
                            bias_up=True, quantile=0.5, seed=1234567890))

# 
sample_size, sample_sizes = find_sample_size(contests, sample_size_function=ss_fn, use_style=use_style, \
                               cvr_list=cvr_list)  
print(f'{sample_size=}\n{sample_sizes=}')

# override for testing
sample_size = 20000
print(sample_size)


## Draw the first sample

In [ ]:
# draw the initial sample
sample_size = 146778
prng = SHA256(seed)
sample = sample_by_index(max_cards, sample_size, prng=prng) # 1-indexed
n_phantom_sample = np.sum(sample > manifest_cards)
print(f'The sample includes {n_phantom_sample} phantom cards.')

In [ ]:
len(cvr_list), manifest_cards, max_cards

In [ ]:
# for comparison audit
# cards_to_retrieve, sample_order, cvr_sample, mvr_phantoms_sample = sample_from_cvrs(cvr_list, manifest, sample)

# for polling audit
cards_to_retrieve, sample_order, mvr_phantoms_sample = sample_from_manifest(manifest, sample)

In [ ]:
# write the sample
write_cards_sampled(sample_file, cards_to_retrieve, print_phantoms=False)

## Read the audited sample data

In [ ]:
# for real data
with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

In [ ]:
# Simulate ballot-polling data for testing/debugging, using the San Francisco IRV contest as an example
mvr_sample = []
for c in cvr_list:
    if c in sample_order.keys():
        mvr_sample.append(c)

for s in set(sample_order.keys()) - set([c.id for c in mvr_sample]):
    inx = np.random.randint(len(cvr_list))
    mvr_sample.append(CVR(id = s, votes = cvr_list[inx].votes, phantom=False))

print(f'simulated sample contains {len(mvr_sample)} mvrs')

## Find measured risks for all assertions

In [ ]:
# prep_comparison_sample(mvr_sample, cvr_sample, sample_order)  # for comparison audit

prep_polling_sample(mvr_sample, sample_order)  # for polling audit
p_max = find_all_p_values(contests, mvr_sample, None, use_style, \
                      risk_function=risk_fn)
print("maximum assertion p-value {}".format(p_max))
done = summarize_status(contests)

In [ ]:
# Log the status of the audit 
write_audit_parameters(log_file, seed, replacement, risk_function, g, max_cards, len(cvr_list), \
                       manifest_cards, phantom_cards, error_rate, contests)

# How many more cards should be audited?

Estimate how many more cards will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* cards already sampled
* the assumption that we will continue to see errors at the same rate observed in the sample

In [ ]:
# Estimate sample size required to confirm the outcome, if errors continue
# at the same rate as already observed.

new_size, sams = new_sample_size(contests, mvr_sample,\
                                 cvr_sample, manifest_type,\
                                 risk_fn, quantile=0.8, reps=100)
new_size

In [ ]:
# augment the sample
# reset the seed
prng = SHA256(seed)
old_sample = sample
sample = sample_by_index(max_cards, new_size, prng=prng)
incremental_sample = np.sort(list(set(sample) - set(old_sample)))
n_phantom_sample = np.sum([cvr_list[i].phantom for i in incremental_sample])
print("The incremental sample includes {} phantom cards.".format(n_phantom_sample))

In [ ]:
cvr_sample_lookup_new, cvr_sample_new, mvr_phantoms_sample_new = \
                sample_from_cvrs(cvr_list, manifest, incremental_sample)
write_cards_sampled(sample_file, cvr_sample_lookup_new, print_phantoms=False)

In [ ]:
# mvr_json should contain the complete set of mvrs, including those in previous rounds

with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots']) 

In [ ]:
# compile entire sample
cvr_sample_lookup, cvr_sample, mvr_phantoms_sample = sample_from_cvrs(cvr_list, manifest, sample)

In [ ]:
# add MVRs for phantoms
mvr_sample = mvr_sample + mvr_phantoms_sample

## Find measured risks for all assertions

In [ ]:
prep_sample(mvr_sample, cvr_sample)
p_max = find_p_values(contests, mvr_sample, cvr_sample, manifest_type, \
                      risk_function= risk_fn)
print("maximum assertion p-value {}".format(p_max))
done = summarize_status(contests)

In [ ]:
# Log the status of the audit 
write_audit_parameters(log_file, seed, replacement, risk_function, g, max_cards, len(cvr_list), \
                       manifest_cards, phantom_cards, error_rate, contests)

In [ ]:
x = np.ones(5)
y = x
y[3]=2
x

In [ ]:
AUDIT_TYPES = (IRV:="IRV", other:="OTHER")

In [ ]:
AUDIT_TYPES

In [1]:
import math
import numpy as np
import scipy as sp
import json
import csv
import warnings
import typing
from numpy import testing
from collections import OrderedDict, defaultdict
from cryptorandom.cryptorandom import SHA256, random, int_from_hash
from cryptorandom.sample import random_permutation
from cryptorandom.sample import sample_by_index

from CVR import CVR
from Audit import Audit, Assertion, Assorter, Contest
from NonnegMean import NonnegMean

In [2]:
rate = 0.01
N = int(10**4)
margin = 0.1
upper_bound = 1
u = 2/(2-margin/upper_bound)
m = (1 - rate*upper_bound/margin)/(2*upper_bound/margin - 1)
one_over = 1/3.8 # 0.5/(2-margin)
clean = 1/1.9    # 1/(2-margin)

AvB = Contest.from_dict({'id': 'AvB',
                     'name': 'AvB',
                     'risk_limit': 0.05,
                     'cards': 10**4,
                     'choice_function': Audit.SOCIAL_CHOICE_FUNCTION.PLURALITY,
                     'n_winners': 1,
                     'candidates': ['Alice','Bob', 'Carol'],
                     'winners': ['Alice'],
                     'audit_type': Audit.AUDIT_TYPE.BALLOT_COMPARISON,
                     'test': NonnegMean.kaplan_markov,
                     'tally': {'Alice': 3000, 'Bob': 2000, 'Carol': 1000},
                     'g': 0.1,
                     'use_style': True
                })
losers = list(set(AvB.candidates)-set(AvB.winners))
AvB.assertions = Assertion.make_plurality_assertions(AvB, winners=AvB.winners, losers=losers)
AvB.find_margins_from_tally()

In [10]:
# first test
for a_id, a in AvB.assertions.items():
    sam_size1 = a.find_sample_size(data=np.ones(10), prefix=True, rate=rate, reps=None, quantile=0.5, seed=1234567890)
    # Kaplan-Markov martingale is \prod (t+g)/(x+g). For x = [1, 1, ...], sample size should be:
    ss1 = math.ceil(np.log(AvB.risk_limit)/np.log((a.test.t+a.test.g)/(1+a.test.g)))
    print(f'{sam_size1} {ss1}')
    assert sam_size1 == ss1
    # For "clean", the term is (1/2+g)/(clean+g); for one_over
    # it is (1/2+g)/(one_over+g). 
    clean = 1/(2-a.margin/a.assorter.upper_bound)
    over = clean/2
    c = (a.test.t+a.test.g)/(clean+a.test.g)
    o = (a.test.t+a.test.g)/(clean/2+a.test.g)
    sam_size2 = a.find_sample_size(data=None, prefix=True, rate=rate, reps=10**3, quantile=0.5, seed=1234567890)
    ss2 = 1+math.ceil(np.log(AvB.risk_limit/o)/np.log(c))
    assert sam_size2 == ss2
    


5 5
x[0:10]=array([0.27777778, 0.55555556, 0.55555556, 0.55555556, 0.55555556,
       0.55555556, 0.55555556, 0.55555556, 0.55555556, 0.55555556])
sam_size2=41 c=0.9152542372881356 o=1.588235294117647 0.045981673147803215 ss2=40
5 5
x[0:10]=array([0.26315789, 0.52631579, 0.52631579, 0.52631579, 0.52631579,
       0.52631579, 0.52631579, 0.52631579, 0.52631579, 0.52631579])
sam_size2=83 c=0.957983193277311 o=1.652173913043478 0.04891057134214332 ss2=82


In [7]:
c=0.957983193277311; o=1.652173913043478

In [9]:
o*c**(sam_size2-2)

0.05105577183960575